<a href="https://colab.research.google.com/github/BootCamp-BMA/colabs/blob/main/dziriBertRandomSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time
import os
import random
from sklearn.metrics import accuracy_score, roc_auc_score

from google.colab import drive
drive.mount('/content/drive')

print("Imported all required libraries")


Mounted at /content/drive
Imported all required libraries


In [ ]:

def load_and_visualize_data(file_path):
    """Load dataset."""
    df = pd.read_csv(file_path)
    return df


In [ ]:

def split_and_tokenize_data(df, tokenizer_name='alger-ia/dziribert', max_length=128, test_size=0.2, random_state=42):
    """Split data into train/test and tokenize using DziriBERT tokenizer."""
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df['news'].tolist(),
        df['label'].tolist(),
        test_size=test_size,
        random_state=random_state
    )
    tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
    train_tokens = tokenizer(train_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
    test_tokens = tokenizer(test_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
    train_labels = torch.tensor(train_labels)
    test_labels = torch.tensor(test_labels)
    return train_tokens, test_tokens, train_labels, test_labels


In [ ]:

def create_dataloaders(train_tokens, test_tokens, train_labels, test_labels, batch_size=16):
    """Create DataLoaders for training and testing."""
    train_data = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], train_labels)
    test_data = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], test_labels)
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_dataloader, test_dataloader


In [ ]:

def setup_model_and_device(model_name='alger-ia/dziribert', num_labels=2, layer_control=None):
    """Set up DziriBERT model and device, with optional layer control."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    model.to(device)
    if layer_control:
        for i, layer in enumerate(model.bert.encoder.layer):
            for param in layer.parameters():
                param.requires_grad = layer_control.get(i, False)
    return model, device


In [ ]:

def setup_training_components(model, train_dataloader, num_epochs=5, lr=2e-5, class_weights=[2.0, 1.0], device=None):
    """Set up optimizer, scheduler, and loss function."""
    optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
    num_training_steps = len(train_dataloader) * num_epochs
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
    class_weights = torch.tensor(class_weights).to(device)
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
    return optimizer, lr_scheduler, criterion


In [ ]:
from tqdm import tqdm  # Import tqdm

def train_model(model, train_dataloader, optimizer, criterion, lr_scheduler, device, epochs=5):
    """Train the model with batch progress tracking per epoch."""
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        # Wrap the batch loop with tqdm
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}", leave=True)
        for batch in progress_bar:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            total_loss += loss.item()
            # Update progress bar with current loss
            progress_bar.set_postfix({'loss': loss.item()})
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{epochs} - Avg Loss: {avg_loss:.4f}")

In [ ]:

def evaluate_model(model, test_dataloader, device):
    """Evaluate the model on test data."""
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)
    return accuracy, roc_auc


In [ ]:

def main():
    """Main function to run experiments with DziriBERT within a 12-hour limit."""
    # Define parameters
    file_path = '/content/drive/MyDrive/arabic_fake_news/FASSILA/cleaned_data.csv'
    save_dir = '/content/drive/MyDrive/arabic_fake_news/FASSILA/'
    results_path = os.path.join(save_dir, "02_experiment_results_DziriBERT.csv")

    batch_sizes = [8, 16, 32]
    num_epochs_list = [3, 5, 7]
    max_lengths = [64, 128, 256]
    learning_rates = [1e-5, 2e-5, 5e-5]
    class_weights_list = [[1.0, 1.0], [2.0, 1.0], [1.0, 2.0]]

    # Load data once
    df = load_and_visualize_data(file_path)
    dataset_size = len(df)

    # System information
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cuda_available = torch.cuda.is_available()
    torch_version = torch.__version__
    transformers_version = transformers.__version__

    # Results storage
    results = []
    save_interval = 10  # Save results every 10 experiments
    max_runtime = 12 * 3600  # 12 hours in seconds
    start_global_time = time.time()

    # Track unique experiment configurations
    tested_configs = set()

    num_layers = 12
    layer_indices = list(range(num_layers))

    iteration = 0
    while True:
        if time.time() - start_global_time > max_runtime:
            print("Time limit reached (12 hours). Stopping experiments.")
            break  # Stop if 12 hours exceeded

        # Randomly select hyperparameters
        batch_size = random.choice(batch_sizes)
        num_epochs = random.choice(num_epochs_list)
        max_length = random.choice(max_lengths)
        lr = random.choice(learning_rates)
        class_weights = random.choice(class_weights_list)

        # Randomly select trainable layers (1 to 6 layers frozen randomly)
        num_trainable = random.randint(1, 6)
        frozen_layers = tuple(sorted(random.sample(layer_indices, num_trainable)))

        # Create a unique key for this configuration
        config_key = (batch_size, num_epochs, max_length, lr, str(class_weights), frozen_layers)

        # Skip if this configuration was tested before
        if config_key in tested_configs:
            continue

        # Mark this configuration as tested
        tested_configs.add(config_key)

        layer_control = {i: (i not in frozen_layers) for i in range(num_layers)}

        start_time = time.time()

        try:
            # Data processing
            train_tokens, test_tokens, train_labels, test_labels = split_and_tokenize_data(df, max_length=max_length)
            train_dataloader, test_dataloader = create_dataloaders(train_tokens, test_tokens, train_labels, test_labels, batch_size)

            # Model setup
            model, device = setup_model_and_device(layer_control=layer_control)

            # Training setup
            optimizer, lr_scheduler, criterion = setup_training_components(model, train_dataloader, num_epochs, lr, class_weights, device)

            # Training
            train_model(model, train_dataloader, optimizer, criterion, lr_scheduler, device, num_epochs)

            # Evaluation
            accuracy, roc_auc = evaluate_model(model, test_dataloader, device)

            elapsed_time = time.time() - start_time

            # Store results for this iteration
            result = {
                'iteration': iteration,
                'dataset_path': file_path,
                'dataset_size': dataset_size,
                'batch_size': batch_size,
                'num_epochs': num_epochs,
                'max_length': max_length,
                'learning_rate': lr,
                'class_weights': str(class_weights),
                'trainable_layers': str(frozen_layers),
                'num_trainable_layers': num_trainable,
                'device': str(device),
                'cuda_available': cuda_available,
                'torch_version': torch_version,
                'transformers_version': transformers_version,
                'execution_time': elapsed_time,
                'accuracy': accuracy,
                'roc_auc': roc_auc
            }
            results.append(result)

            # Minimal printing
            print("Iteration:", iteration)
            print(result)
            print("--------------------------------------------------")

            # Save periodically
            if len(results) % save_interval == 0:
                pd.DataFrame(results).to_csv(results_path, index=False)
            iteration += 1
        except Exception as e:
            print(f"Error in iteration {iteration}: {str(e)}")

    # Final Save
    pd.DataFrame(results).to_csv(results_path, index=False)
    print("\nAll experiments completed!")
    print(f"Final results saved to {results_path}")
    print("Top 5 results sorted by accuracy:")
    print(pd.DataFrame(results).sort_values(by='accuracy', ascending=False).head(5))

# Run the pipeline
main()

Epoch 5/7: 100%|██████████| 482/482 [00:37<00:00, 12.91it/s, loss=0.0861]


Epoch 5/7 - Avg Loss: 0.0805


Epoch 6/7: 100%|██████████| 482/482 [00:37<00:00, 12.93it/s, loss=0.0275]


Epoch 6/7 - Avg Loss: 0.0518


Epoch 7/7: 100%|██████████| 482/482 [00:37<00:00, 12.92it/s, loss=0.0396]


...

Epoch 6/7: 100%|██████████| 241/241 [02:09<00:00,  1.85it/s, loss=0.0871]


Epoch 6/7 - Avg Loss: 0.0662


Epoch 7/7: 100%|██████████| 241/241 [02:09<00:00,  1.85it/s, loss=0.0113]


Epoch 7/7 - Avg Loss: 0.0445
Iteration: 42
{'iteration': 42, 'dataset_path': '/content/drive/MyDrive/arabic_fake_news/FASSILA/cleaned_data.csv', 'dataset_size': 9636, 'batch_size': 32, 'num_epochs': 7, 'max_length': 256, 'learning_rate': 2e-05, 'class_weights': '[2.0, 1.0]', 'trainable_layers': '(7, 9, 11)', 'num_trainable_layers': 3, 'device': 'cuda', 'cuda_available': True, 'torch_version': '2.5.1+cu124', 'transformers_version': '4.48.3', 'execution_time': 927.8011448383331, 'accuracy': 0.766597510373444, 'roc_auc': 0.7570035726132279}
--------------------------------------------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alger-ia/dziribert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/7:  84%|████████▍ | 203/241 [01:53<00:21,  1.78it/s, loss=0.463]